# Objective: Use AFLite to greedily solve for $\text{arg min}_{S \subset \mathcal{D}, ~|S| \geq n}\mathcal{R}(\Phi, ~S, ~\mathcal{M})$

### 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from transformers import GPT2TokenizerFast, DataCollatorWithPadding, set_seed
import torch
from torch.nn.functional import one_hot
import copy
import numpy as np
import sys
sys.path.append('..')
from utils_ import tokenize, train_classifier, predict, select_k
import pickle
import itertools
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
disable_caching()

### 2. Pre-Processing
- Get SNLI Dataset (Train fold) and shuffle it using the same seed as used for obtaining GPT-2 based Feature Representation (see notebook [Filtering_Part1.ipynb](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/Filtering_Part1.ipynb))
- Remove instances without gold standard labels, i.e., label = -1
- One-hot encoding for labels
- Partition data 10%/90%; use the 90% as `train`
- Tokenise train

In [2]:
snli_train = load_dataset('snli', split = 'train').shuffle(seed = 42)
snli_train = snli_train.filter(lambda x: x['label'] != -1).map( \
    lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
    batched = True)
train = snli_train.select(range(int(len(snli_train)/10), len(snli_train)))

Reusing dataset snli (/home/shana92/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

In [3]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2-medium", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [4]:
# tokenize data
train = train.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
len_bef_exclusion = len(train)

# exclude instances with > 128 tokens
train = train.filter(lambda x: x['exclude'] == False)
len_aft_exclusion = len(train)

# print message if instances were in fact excluded
if len_bef_exclusion - len_aft_exclusion > 0:
    
    print(f'{len_bef_exclusion - len_aft_exclusion} ' + \
          f'({(len_bef_exclusion/len_aft_exclusion - 1)*100:>2f}%) sequences excluded')

  0%|          | 0/494431 [00:00<?, ?ex/s]

  0%|          | 0/495 [00:00<?, ?ba/s]

In [5]:
# keep only needed columns, set data format to PyTorch
train.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])

### 3. Set up inputs for AFLite

In [6]:
# load in the feature representation, Phi, with linear layer attached
model = torch.load('feature_rep.pth')

# move model to CPU
model.to('cpu')

# freeze all layers except the last
num_layers = sum(1 for _ in model.parameters())
for idx, param in enumerate(model.parameters()):
    
    if idx != num_layers - 1:
        
        # freeze
        param.requires_grad = False

In [7]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

In [8]:
# hyper-parameters - constrained by training time available
m = 30
n = 195000
t = 50000
k = 100000
tau = 0.75
AFLite_seeds = [0, 1, 2, 3, 4]

In [9]:
# hyper-parameters for model training within AFLite implementation
batch_size = 128 # constrained by GPU memory
lr = 1e-5 # set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108

### 4.  AFLite Procedure

In [10]:
# set up containers to collect outputs
filtered_datasets = {}
removed_idx = {x: '' for x in AFLite_seeds}

# begin procedure
for seed in AFLite_seeds:
    
    # first step of AFLite; initialise S
    S = copy.deepcopy(train)
    
    # initialise iteration index
    it_idx = 0
    
    while len(S) > n:
        
        # update iteration index
        it_idx += 1
        
        # initialise multiset for Out-Of-Sample predictions
        E = {x: [] for x in range(len(S))}

        for j in range(m):
            
            # randomly partition S into (S\T_j, T_j) s.t. |S\T_j| = t
            tr_idx = set(np.random.default_rng(j).choice(np.arange(len(S)), t, replace = False))
            te_idx = set(range(len(S))) - tr_idx
            tr, te = S.select(tr_idx), S.select(te_idx)
            print(f'Seed {seed} - Iteration {it_idx} - Model {j + 1} - Begin')
                        
            # train classifier on S\T_j, i.e. tr
            classifier = copy.deepcopy(model)
            dataloader = torch.utils.data.DataLoader(tr, batch_size=batch_size, \
                                 shuffle=True, collate_fn=data_collator)
            optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()), lr = lr)
            trained_classifier = train_classifier(classifier, dataloader, optimizer, device)
            
            # for all instances i in T_j, add predictions to E(i)
            te_dataloader = torch.utils.data.DataLoader(te, batch_size=batch_size, collate_fn=data_collator)
            preds = predict(trained_classifier, te_dataloader, device)
            print(f'Seed {seed} - Iteration {it_idx} - Model {j + 1} - Done')
            
            for pred_idx, data_idx in enumerate(te_idx): # there are as many predictions as test instances
                
                E[data_idx] += [preds[pred_idx]]
        
        # for all instances in S, compute predictability score
        # in the corner case that there are no predictions for an instance, we do not filter it out
        lengths = torch.tensor([len(x) if len(x) > 0 else 1 for x in E.values()])
        preds_padded = torch.tensor(list(itertools.zip_longest(*E.values(), fillvalue=-1))).transpose(0, 1)
        labels = torch.repeat_interleave(S['label'].argmax(1), max(lengths)).reshape(preds_padded.size())

        pred_matches = torch.eq(preds_padded, labels)
        pred_match_totals = torch.sum(pred_matches, axis = 1)
        pred_scores = pred_match_totals / lengths
        
        # select up to k instances with the highest predictability scores subject to score >= tau
        selected_idx = select_k(pred_scores, tau, k, seed)
        
        if selected_idx.shape[0] > 0:
        
            # cache instances selected for removal
            removed_idx[seed] += ',' + ','.join([str(idx) for idx in selected_idx])

            # filter out selected instances
            S = S.select(set(range(len(S))) - set(selected_idx))
        
        # early stopping
        elif selected_idx.shape[0] < k:
            
            break
    
    # cache file
    filtered_datasets[seed] = S
    
    # print number of instances in S, for creating random baseline
    print(f'Number of instances in S (seed {seed}): {len(S)}')
    
# write out list of removed indices for further analysis
with open('removed_idx.pkl', 'wb') as f:
    pickle.dump(removed_idx, f)

Seed 0 - Iteration 1 - Model 1 - Begin


  0%|          | 0/391 [00:00<?, ?it/s]

loss: 0.490815  [    0/50000]
loss: 0.462348  [ 4992/50000]
loss: 0.483250  [ 9984/50000]
loss: 0.501971  [14976/50000]
loss: 0.445378  [19968/50000]
loss: 0.507128  [24960/50000]
loss: 0.435846  [29952/50000]
loss: 0.411889  [34944/50000]
loss: 0.481307  [39936/50000]
loss: 0.452662  [44928/50000]
loss: 0.496749  [31200/50000]
Epoch average loss: 0.4680524170398712


  0%|          | 0/391 [00:00<?, ?it/s]

loss: 0.488783  [    0/50000]
loss: 0.467965  [ 4992/50000]
loss: 0.482960  [ 9984/50000]
loss: 0.464695  [14976/50000]
loss: 0.479309  [19968/50000]
loss: 0.460052  [24960/50000]
loss: 0.459742  [29952/50000]
loss: 0.501513  [34944/50000]
loss: 0.476872  [39936/50000]
loss: 0.534709  [44928/50000]
loss: 0.460403  [31200/50000]
Epoch average loss: 0.46620702743530273


...

  0%|          | 0/391 [00:00<?, ?it/s]

loss: 0.593714  [    0/50000]
loss: 0.508670  [ 4992/50000]
loss: 0.504293  [ 9984/50000]
loss: 0.544874  [14976/50000]
loss: 0.628349  [19968/50000]
loss: 0.530994  [24960/50000]
loss: 0.589618  [29952/50000]
loss: 0.530466  [34944/50000]
loss: 0.570835  [39936/50000]
loss: 0.557941  [44928/50000]
loss: 0.562434  [31200/50000]
Epoch average loss: 0.5550875067710876
Done!


  0%|          | 0/1910 [00:00<?, ?it/s]

Seed 4 - Iteration 3 - Model 30 - Done
Number of instances in S (seed 4): 194431
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/shana92/.conda/envs/text/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
  File "/tmp/ipykernel_19498/2042503834.py", line 79, in <cell line: 79>
    with open('removed_idx.pkl', 'wb') as f:
PermissionError: [Errno 13] Permission denied: 'removed_idx.pkl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shana92/.conda/envs/text/lib/python3.8/site-packages/pygments/styles/__init__.py", line 78, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shana92/.conda/envs/text/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1992, in showtraceback
  File "/home/shana92/.conda/envs/text/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback

In [12]:
# write out list of removed indices for further analysis
with open('removed_idx.pkl', 'wb') as f:
    pickle.dump(removed_idx, f)